In [2]:
!pip install alpaca_trade_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 19.9 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.7
    Uninstalling msgpa

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import alpaca_trade_api as tradeapi

# API keys
alpaca_api_key = "PKJYB527YAQVS4J907RJ"
alpaca_secret_key = "zb5bvTpu8cWaKIgAbDCESouV9MX0qn4RXiQzS5n8"

# Create the Alpaca API object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

# Set the tickers and timeframe
tickers = ["XOM"]
timeframe = "1Day"

# Set the date range
today = pd.Timestamp.today(tz="America/New_York").normalize().isoformat()

start = pd.Timestamp("2000-02-07", tz="America/New_York").isoformat()
end = pd.Timestamp("2023-12-13", tz="America/New_York").isoformat()

# Get the data
df = alpaca.get_bars(tickers, timeframe, start=start, end=end).df

# Drop the 'symbol' column and add shifted close column
df = df.drop(columns=['symbol'])

# Function to create the signal
def signal_creator_5(row):
    percent_diff = (row['close'] - row['shifted_close_5']) / row['shifted_close_5']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

def signal_creator_15(row):
    percent_diff = (row['close'] - row['shifted_close_15']) / row['shifted_close_15']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

def signal_creator_30(row):
    percent_diff = (row['close'] - row['shifted_close_30']) / row['shifted_close_30']
    if percent_diff > 0.03:
        return 1
    elif percent_diff < -0.03:
        return -1
    else:
        return 0

# 5 day shift
df['shifted_close_5'] = df['close'].shift(periods=5)
df['signal_5'] = 0
df['signal_5'] = df.apply(signal_creator_5, axis=1)

# 15 day shift
df['shifted_close_15'] = df['close'].shift(periods=15)
df['signal_15'] = 0
df['signal_15'] = df.apply(signal_creator_15, axis=1)

# 30 day shift
df['shifted_close_30'] = df['close'].shift(periods=30)
df['signal_30'] = 0
df['signal_30'] = df.apply(signal_creator_30, axis=1)

# Columns and NA drops to prevent data leakege
display(df.tail(10))

,close,high,low,trade_count,open,volume,vwap,shifted_close_5,signal_5,shifted_close_15,signal_15,shifted_close_30,signal_30
timestamp,,,,,,,,,,,,,
2023-11-30 05:00:00+00:00,102.74,104.4778,101.810,210666,103.11,29254689,102.875059,104.01,0,102.93,0,112.95,-1
2023-12-01 05:00:00+00:00,102.99,104.2200,102.220,164627,102.50,17845278,103.300795,104.57,0,102.96,0,113.02,-1
2023-12-04 05:00:00+00:00,102.43,103.3600,101.590,183900,102.13,21523942,102.555546,103.96,0,103.75,0,111.08,-1
2023-12-05 05:00:00+00:00,100.44,102.3400,100.290,197925,102.27,22250251,101.021919,103.90,-1,104.84,-1,109.45,-1
2023-12-06 05:00:00+00:00,99.11,100.7200,98.340,275143,100.00,31154792,99.278750,102.34,-1,104.29,-1,108.39,-1
2023-12-07 05:00:00+00:00,98.42,100.8350,98.090,259510,99.86,29990727,98.969257,102.74,-1,103.66,-1,108.59,-1
2023-12-08 05:00:00+00:00,99.55,99.7867,98.750,196370,99.02,22070842,99.355056,102.99,-1,102.46,0,107.60,-1
2023-12-11 05:00:00+00:00,99.62,99.8600,98.580,188728,99.80,22252250,99.353079,102.43,0,104.96,-1,105.55,-1
2023-12-12 05:00:00+00:00,98.05,98.6800,97.480,209533,98.68,26084064,98.065970,100.44,0,104.50,-1,105.88,-1


In [4]:

df.dropna(inplace=True)
df.drop(columns=['shifted_close_5','shifted_close_15','shifted_close_30'],inplace=True)
display(df.tail(50))

# Separate features and target
features = df.drop(columns = ['signal_5', 'signal_15', 'signal_30'], axis=1)
target = df.drop(columns = ['close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap','signal_15', 'signal_30'], axis=1)
display(target)


,close,high,low,trade_count,open,volume,vwap,signal_5,signal_15,signal_30
timestamp,,,,,,,,,,
2023-10-04 04:00:00+00:00,111.50,114.0900,110.5018,202537,114.05,19622227,111.867086,-1,-1,1
2023-10-05 04:00:00+00:00,108.99,110.9000,108.7000,202709,109.80,20128784,109.333986,-1,-1,0
2023-10-06 04:00:00+00:00,107.17,108.2700,105.1100,263692,105.73,25367468,106.806588,-1,-1,0
2023-10-09 04:00:00+00:00,110.92,112.0700,109.5200,232884,110.44,22369888,110.892577,-1,-1,0
2023-10-10 04:00:00+00:00,110.45,111.4500,110.1200,150655,111.02,13891954,110.755690,-1,-1,0
2023-10-11 04:00:00+00:00,106.49,107.1700,104.8300,583462,106.96,57939025,105.936671,-1,-1,-1
2023-10-12 04:00:00+00:00,106.47,107.8400,106.0900,301121,107.59,31182959,106.921382,0,-1,-1
2023-10-13 04:00:00+00:00,109.87,110.2000,107.8400,292833,108.03,30290330,109.319037,0,-1,0
2023-10-16 04:00:00+00:00,109.95,110.6800,109.1300,188300,110.50,18451997,109.871895,0,-1,-1


,signal_5
timestamp,
2016-02-17 05:00:00+00:00,0
2016-02-18 05:00:00+00:00,1
2016-02-19 05:00:00+00:00,1
2016-02-22 05:00:00+00:00,0
2016-02-23 05:00:00+00:00,0
...,...
2023-12-07 05:00:00+00:00,-1
2023-12-08 05:00:00+00:00,-1
2023-12-11 05:00:00+00:00,0


In [5]:
# Scale the features
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

In [6]:
# Function to create dataset for LSTM
def create_dataset(X, y, time_steps=5):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps), :])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [7]:
# Creating the dataset
time_steps = 5
X, y = create_dataset(scaled_features, target.values, time_steps)

In [8]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=13)

In [9]:
model_5 = Sequential()

In [10]:
# model_15 = Sequential()

In [11]:
# model_30 = Sequential()

In [12]:
# Building the LSTM model
def builder(model):
  model.add(LSTM(150, return_sequences=True, input_shape=(time_steps, X.shape[2])))
  model.add(Dropout(0.2))
  model.add(LSTM(75, return_sequences=False))
  model.add(Dense(50,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(25,activation='relu'))
  model.add(Dense(1,activation='linear'))

In [13]:
builder(model_5)

In [ ]:
# builder(model_15)

In [ ]:
# builder(model_30)

In [14]:
# Compiling the model
model_5.compile(optimizer='adam', loss='mean_squared_error')
# model_15.compile(optimizer='adam', loss='mean_squared_error')
# model_30.compile(optimizer='adam', loss='mean_squared_error')


In [15]:
# Training the model
model_5.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
50/50 [==============================] - 6s 16ms/step - loss: 0.3370
Epoch 2/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3365
Epoch 3/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3367
Epoch 4/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3364
Epoch 5/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3359
Epoch 6/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3367
Epoch 7/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3355
Epoch 8/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3364
Epoch 9/50
50/50 [==============================] - 1s 25ms/step - loss: 0.3356
Epoch 10/50
50/50 [==============================] - 1s 27ms/step - loss: 0.3363
Epoch 11/50
50/50 [==============================] - 1s 25ms/step - loss: 0.3362
Epoch 12/50
50/50 [==============================] - 1s 15ms/step - loss: 0.3356
Epoch 13/50
50/50 [==================

In [ ]:
# model_15.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# model_30.fit(X_train, y_train, epochs=50, batch_size=32)

In [16]:
# Model Evaluation
def eval(model):
  test_loss = model.evaluate(X_test, y_test)
  print(f'Test Loss: {test_loss}')


In [17]:
eval(model_5)

13/13 [==============================] - 1s 6ms/step - loss: 0.3585
Test Loss: 0.3585048317909241


In [18]:
# eval(model_15)
aplication_size = int(len(X)*0.9) # 20% of the data

X_train_app, X_test_app = X[:aplication_size], X[aplication_size:]
y_train_app, y_test_app = y[:aplication_size], y[aplication_size:]

# Calculate the split point


In [ ]:
# eval(model_30)

In [19]:
# Making Predictions
predictions_5 = model_5.predict(X_test_app)
print(predictions_5)

7/7 [==============================] - 1s 8ms/step
[[0.04582744]
 [0.04623478]
 [0.04611575]
 [0.0457437 ]
 [0.04564093]
 [0.0450349 ]
 [0.04422955]
 [0.04422105]
 [0.0435329 ]
 [0.04334715]
 [0.04325305]
 [0.04313286]
 [0.04462006]
 [0.04505352]
 [0.04489309]
 [0.04494508]
 [0.04471996]
 [0.0440218 ]
 [0.04460885]
 [0.04525231]
 [0.04523663]
 [0.04510008]
 [0.04598043]
 [0.04674371]
 [0.04747903]
 [0.04801458]
 [0.04756548]
 [0.04757643]
 [0.04739796]
 [0.0472598 ]
 [0.0469696 ]
 [0.04677311]
 [0.04706809]
 [0.0469523 ]
 [0.04733658]
 [0.04752046]
 [0.04747575]
 [0.04765759]
 [0.04743144]
 [0.0473887 ]
 [0.04719642]
 [0.04673094]
 [0.04643444]
 [0.0459587 ]
 [0.04566643]
 [0.0461024 ]
 [0.04592802]
 [0.04567235]
 [0.04458556]
 [0.0440454 ]
 [0.04403692]
 [0.04385992]
 [0.04388606]
 [0.04361684]
 [0.04402851]
 [0.04426256]
 [0.04411807]
 [0.0441699 ]
 [0.04422463]
 [0.04436171]
 [0.04411137]
 [0.04317519]
 [0.04267578]
 [0.04288584]
 [0.04376492]
 [0.04434954]
 [0.0444889 ]
 [0.0445582

In [20]:
# predictions_15 = model_15.predict(X_test)
# print(confusion_matrix(y_test, predictions_5))
y_test_app

array([[ 0],
       [ 0],
       [ 0],
       [ 0],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 1],
       [ 0],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [-1],
       [-1],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [-1],
       [-1],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 1],
       [ 0],
       [ 0],
       [ 0],
       [-1],
       [ 0],
       [ 0],
       [-1],
       [ 0],
       [ 0],
       [-1],

In [ ]:
# predictions_30 = model_30.predict(X_test)


In [21]:
# Function to interpret predictions
def interpret_prediction(prediction):
    if prediction > 0.3:
        return 1  # Buy
    elif prediction < -0.2:
        return -1  # Sell
    else:
        return 0  # Hold

#interpreted_signals = [interpret_prediction(p[0]) for p in predictions]

# Example: Print first 10 decisions




In [22]:
# results_5 = [interpret_prediction(p[0]) for p in predictions_5]
# print(results_5[:1])
pred_result = [interpret_prediction(p[0]) for p in predictions_5]
confusion_matrix(y_test_app, pred_result)

array([[  0,  40,   0],
       [  0, 120,   0],
       [  0,  37,   0]])

In [23]:
# results_15 = [interpret_prediction(p[0]) for p in predictions_15]
# print(results_15[:-25])
print(classification_report(y_test_app, pred_result))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        40
           0       0.61      1.00      0.76       120
           1       0.00      0.00      0.00        37

    accuracy                           0.61       197
   macro avg       0.20      0.33      0.25       197
weighted avg       0.37      0.61      0.46       197



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# results_30 = [interpret_prediction(p[0]) for p in predictions_30]
# print(results_30[:-25])
pred_result[-1]

0

In [35]:
# Scaling for inverse transform
scaler = StandardScaler()
scaler.fit(features)


StandardScaler()

In [39]:
# Scale the data
scaled_feat = scaler.transform(features)

In [40]:
# Inverse transform
original_data = scaler.inverse_transform(scaled_feat)

In [41]:
print("Original Data:\n", features)
print("Inverse Transformed Data:\n", original_data)

Original Data:
                            close      high     low  trade_count   open  \
timestamp                                                                
2016-02-17 05:00:00+00:00  82.00   82.7750  81.160       113916  81.65   
2016-02-18 05:00:00+00:00  82.45   83.0600  81.860       108206  82.94   
2016-02-19 05:00:00+00:00  82.50   82.5500  80.910        88061  82.20   
2016-02-22 05:00:00+00:00  82.39   83.4400  82.200        91572  83.19   
2016-02-23 05:00:00+00:00  81.23   82.3700  80.879        74235  81.99   
...                          ...       ...     ...          ...    ...   
2023-12-07 05:00:00+00:00  98.42  100.8350  98.090       259510  99.86   
2023-12-08 05:00:00+00:00  99.55   99.7867  98.750       196370  99.02   
2023-12-11 05:00:00+00:00  99.62   99.8600  98.580       188728  99.80   
2023-12-12 05:00:00+00:00  98.05   98.6800  97.480       209533  98.68   
2023-12-13 05:00:00+00:00  98.92   99.0300  97.655       166290  98.04   

                     